# mobilenets with tensorflow keras API

In [ ]:
# https://deeplizard.com/learn/video/OO4HD-1wRN8
# https://deeplizard.com/learn/video/FNqp4ZY0wDY
# https://deeplizard.com/learn/video/Zrt76AIbeh4

# Face recognition using Google's pretrained mobilenet model

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from IPython.display import display
from PIL import Image
from tensorflow.keras.models import load_model
%matplotlib inline

In [ ]:
import pickle

In [ ]:
#from custom_callbacks import ModelCheckpointEnhanced
#from custom_callbacks import LearningRateSchedulerPerBatch
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ModelCheckpoint


# Organize data into train, valid, test dirs

In [ ]:
faces_directory=r'F:\f\face\face recognition\mobilenet'
os.chdir(faces_directory)

In [ ]:
if os.path.isdir('train/0/') is False:
    os.mkdir('train')
    os.mkdir('valid')
    os.mkdir('test')

    for i in range(0, 10):
        shutil.move(f'{i}', 'train')
        os.mkdir(f'valid/{i}')
        os.mkdir(f'test/{i}')

        valid_samples = random.sample(os.listdir(f'train/{i}'), 550)
        for j in valid_samples:
            shutil.move(f'train/{i}/{j}', f'valid/{i}')

        test_samples = random.sample(os.listdir(f'train/{i}'), 100)
        for k in test_samples:
            shutil.move(f'train/{i}/{k}', f'test/{i}')

# processing the data

In [ ]:
train_path =os.path.join(f'{faces_directory}/train' )
valid_path = os.path.join(f'{faces_directory}/valid' )
test_path = os.path.join(f'{faces_directory}/test' )

In [ ]:

data_generator=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)
train_batches = data_generator.flow_from_directory(directory=train_path, target_size=(224,224), batch_size=10)
valid_batches = data_generator.flow_from_directory(directory=valid_path, target_size=(224,224), batch_size=10)
test_batches = data_generator.flow_from_directory(directory=test_path, target_size=(224,224), batch_size=10, shuffle=False)

# downloading the model

In [ ]:
#original model
mobile = tf.keras.applications.mobilenet.MobileNet()
print(mobile.summary())

## creating the new model

In [ ]:
x = mobile.layers[-6].output
output = Dense(units=4, activation='softmax')(x)
model = Model(inputs=mobile.input, outputs=output)
for layer in model.layers[:-30]:
    layer.trainable = False #freeze top layers except the last 30
model.summary()

## callbacks

In [ ]:

filepath=f'{faces_directory}/weights.best3.hdf5'
#lr_decay_callback = LearningRateSchedulerPerBatch(lambda step: ((learning_rate - min_learning_rate)* 
                         #                                       decay_rate ** step 
                           #                                     + min_learning_rate))
def scheduler(epoch, lr):
    if epoch < 13:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
lr_decay_callback = LearningRateScheduler(scheduler)

checkpoint = ModelCheckpoint(filepath, 
                                 monitor='val_accuracy', 
                                 verbose=1, 
                                 save_best_only=True, 
                                     mode='auto')
 
                                

early_stopping= EarlyStopping(monitor="val_accuracy",min_delta=0.005, patience=5, verbose=1,mode="auto",baseline=None,restore_best_weights=True)
                            
                            
                                
                               
                               

callbacks_list = [checkpoint,lr_decay_callback,early_stopping]

## compiling and fitting

In [ ]:
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

if os.path.isfile(f'{faces_directory}/face_classifier.h5') is False:
    history= model.fit(x=train_batches,
              validation_data=valid_batches,
              epochs=40,
              callbacks=callbacks_list,
              verbose=2
              )
    model.save(os.path.join(f'{faces_directory}/face_classifier.h5'))

# evaluating the transfer-learned model

In [ ]:
new_model=load_model(f'{faces_directory}/face_classifier2.h5')
test_labels = test_batches.classes
predictions = new_model.predict(x=test_batches, steps=len(test_batches), verbose=0)


## plotting predictions in a confusion matrix

In [ ]:
cm = confusion_matrix(y_true=test_labels, y_pred=predictions.argmax(axis=1))
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
print(test_batches.class_indices)
cm_plot_labels = ['0','1','2','3','4','5','6','7','8','9']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')
plt.show()